<a href="https://colab.research.google.com/github/Josphat-Malombe/deep_learning/blob/main/model_009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle --quiet
!pip install opendatasets --quiet
import opendatasets as od
url="https://www.kaggle.com/datasets/andrewmvd/animal-faces"
od.download(url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: josphatmalombe
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/animal-faces


In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import transforms
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image

In [16]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [17]:
image_path=[]
labels=[]

for i in os.listdir("/content/animal-faces/afhq"):
  for label in os.listdir(f"/content/animal-faces/afhq/{i}"):
    for image in os.listdir(f"/content/animal-faces/afhq/{i}/{label}"):
      image_path.append(f"/content/animal-faces/afhq/{i}/{label}/{image}")
      labels.append(label)
data_df=pd.DataFrame({"image_path":image_path,"label":labels})
data_df.head()



,image_path,label
0,/content/animal-faces/afhq/train/dog/pixabay_d...,dog
1,/content/animal-faces/afhq/train/dog/flickr_do...,dog
2,/content/animal-faces/afhq/train/dog/pixabay_d...,dog
3,/content/animal-faces/afhq/train/dog/pixabay_d...,dog
4,/content/animal-faces/afhq/train/dog/pixabay_d...,dog


In [18]:
#shuffling the data to avoid bias
data_df=data_df.sample(frac=1,random_state=42).reset_index(drop=True)
#taking 70% of data assigning it to training
train=data_df.sample(frac=0.7)
#remaining 30% assigned to test
test=data_df.drop(train.index)
#50% of the test data assigned to validation
val=test.sample(frac=0.5)
test=test.drop(val.index)

print(train.shape,val.shape,test.shape)

(11291, 2) (2420, 2) (2419, 2)


In [19]:
label_encoder=LabelEncoder()
label_encoder.fit(data_df['label'])

train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32)
])

val_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32)
])



In [20]:
class dataset(Dataset):
  def __init__(self,dataframe,transform=None):
    self.dataframe=dataframe
    self.transform=transform
    self.labels=torch.Tensor(label_encoder.transform(self.dataframe['label'])).type(torch.LongTensor).to(device)

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, idx):
    image_path=self.dataframe.iloc[idx,0]
    label=self.labels[idx]
    image=Image.open(image_path).convert("RGB")

    image=self.transform(image).to(device)
    return image,label



In [21]:
train_dataset=dataset(train,train_transform)
test_dataset=dataset(test,val_transform)
val_dataset=dataset(val,val_transform)

print(len(train_dataset),len(test_dataset),len(val_dataset))

11291 2419 2420


In [22]:
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=32,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=32,shuffle=True)


In [25]:
class ClassifiModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv_block1=nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.conv_block2=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.flatten=nn.Flatten()
    self.linear_block=nn.Sequential(
        nn.Linear(128*56*56,512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512,3)
    )
  def forward(self,x):
    x=self.conv_block1(x)
    x=self.conv_block2(x)
    x=self.flatten(x)
    x=self.linear_block(x)
    return x


In [26]:
model=ClassifiModel().to(device)

In [27]:
from torchsummary import summary
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          18,496
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
          Flatten-11               [-1, 401408]               0
           Linear-12                  [-1, 512]     205,521,408
             ReLU-13                  [-1, 512]               0
          Dropout-14                  [

In [28]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [30]:
epochs=10


for epoch in range(epochs):
  model.train()
  train_loss,train_acc=0,0

  for data in train_dataloader:
    inputs,label=data
    inputs,label=inputs.to(device),label.to(device)
    optimizer.zero_grad()
    outputs=model(inputs)
    loss=criterion(outputs,label)
    train_loss+=loss.item()
    acc=(torch.argmax(outputs, dim=1)==label).sum().item()
    train_acc+=acc
    loss.backward()
    optimizer.step()

  model.eval()
  val_loss,val_acc=0,0
  with torch.no_grad():
    for data in val_dataloader:
      inputs,labels=data
      inputs,labels=inputs.to(device),labels.to(device)
      outputs=model(inputs)
      loss=criterion(outputs,labels)
      val_loss+=loss.item()
      acc=(torch.argmax(outputs,dim=1)==labels).sum().item()
      val_acc+=acc

  train_loss/=len(train_dataloader)
  val_loss/=len(val_dataloader)

  train_acc=(train_acc/len(train_dataset))*100
  val_acc=(val_acc/len(val_dataset))*100

  print(f"Epoch:{epoch+1}/{epochs} | Train loss:{train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val loss: {val_loss:.4f} | val acc: {val_acc:.2f}%")












Epoch:1/10 | Train loss:0.8428 | Train Acc: 57.15% | Val loss: 0.4119 | val acc: 83.06%
Epoch:2/10 | Train loss:0.4153 | Train Acc: 83.52% | Val loss: 0.3469 | val acc: 86.24%
Epoch:3/10 | Train loss:0.3571 | Train Acc: 85.55% | Val loss: 0.2910 | val acc: 88.43%
Epoch:4/10 | Train loss:0.3177 | Train Acc: 87.62% | Val loss: 0.2878 | val acc: 88.76%
Epoch:5/10 | Train loss:0.2809 | Train Acc: 88.80% | Val loss: 0.2539 | val acc: 90.33%
Epoch:6/10 | Train loss:0.2651 | Train Acc: 89.50% | Val loss: 0.2266 | val acc: 91.12%
Epoch:7/10 | Train loss:0.2722 | Train Acc: 89.48% | Val loss: 0.2342 | val acc: 91.07%
Epoch:8/10 | Train loss:0.2360 | Train Acc: 90.66% | Val loss: 0.2271 | val acc: 91.24%
Epoch:9/10 | Train loss:0.2217 | Train Acc: 91.61% | Val loss: 0.2014 | val acc: 92.85%
Epoch:10/10 | Train loss:0.2026 | Train Acc: 92.25% | Val loss: 0.1950 | val acc: 92.93%
